In [ ]:
import urllib.request
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Define column names to parse
COLUMNS = [
    "stationId", "observationTime", "NFDRType", "fuelModelType",
    "thousandHR_TL_FuelMoisture", "ignitionComponent",
    "spreadComponent", "energyReleaseComponent", "burningIndex"
]

# Functions
def download_csv(url, filename):
    """Download a CSV file from a URL."""
    try:
        urllib.request.urlretrieve(url, filename)
        print(f"File downloaded successfully: {filename}")
    except Exception as e:
        print(f"Error downloading file: {e}")
        exit()

def load_and_process_csv(filename):
    """Load a CSV file into a DataFrame and process it."""
    try:
        df = pd.read_csv(filename, usecols=COLUMNS, dtype={
            "stationId": "category",
            "NFDRType": "category",
            "fuelModelType": "category",
            "thousandHR_TL_FuelMoisture": "float32",
            "ignitionComponent": "float32",
            "spreadComponent": "float32",
            "energyReleaseComponent": "float32",
            "burningIndex": "float32"
        })
        print(f"Data loaded successfully with shape: {df.shape}")
    except Exception as e:
        print(f"Error loading data: {e}")
        exit()

    # Convert observationTime to datetime
    df['DateTime'] = pd.to_datetime(df['observationTime'], errors='coerce')
    # Drop invalid rows
    df = df.dropna(subset=['DateTime'])
    # Calculate Julian Day
    df['JDay'] = df['DateTime'].dt.dayofyear
    return df

def calculate_daily_stats(df):
    """Calculate daily max and average statistics for the selected component."""
    daily_max = df.groupby('JDay').max(numeric_only=True).reset_index()
    daily_avg = df.groupby('JDay').mean(numeric_only=True).reset_index()
    return daily_max, daily_avg

def calculate_quantiles(df, component):
    """Calculate quantiles for the selected component."""
    return df[component].quantile([0.6, 0.8, 0.9, 0.97, 0.999])

def plot_erc(historical_max, historical_avg, current_max, forecast_max, quantiles, station_id, component):
    """Plot historical, current year, and forecast data with quantile highlights."""
    plt.figure(figsize=(12, 8))

    # Plot historical max and average
    plt.plot(historical_max['JDay'], historical_max[component], color='red', label='Max', linewidth=0.8)
    plt.plot(historical_avg['JDay'], historical_avg[component], color='grey', label='Average', linewidth=0.8)

    # Plot current year max
    plt.plot(current_max['JDay'], current_max[component], color='blue', label='Current', linewidth=1.2)

    # Plot forecast max (green)
    plt.plot(forecast_max['JDay'], forecast_max[component], color='green', label='Forecast', linewidth=1.2)

    # Calculate the maximum value for the chart
    y_max = max(historical_max[component].max(),
                current_max[component].max(),
                forecast_max[component].max())

    # Extend quantile shading to the top of the chart
    ax = plt.gca()
    ax.axhspan(0, quantiles[0.6], color='green', alpha=0.2)
    ax.axhspan(quantiles[0.6], quantiles[0.8], color='lightgreen', alpha=0.2)
    ax.axhspan(quantiles[0.8], quantiles[0.9], color='tan', alpha=0.2)
    ax.axhspan(quantiles[0.9], quantiles[0.97], color='orange', alpha=0.2)
    ax.axhspan(quantiles[0.97], quantiles[0.999], color='red', alpha=0.2)
    ax.axhspan(quantiles[0.999], y_max * 1.1, color='darkred', alpha=0.2)

    # Convert Julian Day (JDay) to calendar day labels
    start_of_year = datetime(2025, 1, 1)
    calendar_days = [start_of_year + timedelta(days=int(jday) - 1) for jday in range(1, 366)]
    calendar_day_labels = [day.strftime('%b %d') for day in calendar_days]

    # Set x-axis ticks to 15-day intervals
    tick_intervals = range(1, 366, 15)  # 15-day intervals
    ax.set_xticks(tick_intervals)
    ax.set_xticklabels([calendar_days[jday - 1].strftime('%b %d') for jday in tick_intervals], rotation=45)

    # Set axis limits and labels
    plt.xlabel("Calendar Day")
    plt.ylabel(component)
    plt.title(f"{component} - Station {station_id}")
    plt.grid(True)
    plt.xlim(1, 365)
    plt.ylim(0, y_max * 1.1)  # Extend slightly beyond the maximum value for better visibility
    
    # Add percentile lines with labels
    for q, label, style in zip([0.999, 0.97, 0.9, 0.8, 0.6],
                               ["99.9%", "97%", "90%", "80%", "60%"],
                               [":", "--", "-.", "-", "-"]):
        plt.hlines(quantiles[q], 0, 365, label=f"{label} - {quantiles[q]:.1f}", linestyle=style, linewidth=1.5, color='black')

    # Ensure all labels are included in the legend
    plt.legend(loc="upper left", fontsize="small")

    # Show the chart
    plt.show()

# Main Script
if __name__ == "__main__":
    # User input for station IDs
    station_ids = input("Enter station IDs separated by commas, or provide a CSV file path: ").strip()
    
    # Check if the input is a CSV file or direct station IDs
    if station_ids.endswith(".csv"):
        # Load station IDs from CSV
        station_df = pd.read_csv(station_ids)
        station_ids = station_df['stationId'].tolist()
    else:
        # Split direct input into a list
        station_ids = [station_id.strip() for station_id in station_ids.split(",")]

    # Get today's date and 7 days from today
    today = datetime.utcnow()
    seven_days_from_today = today + timedelta(days=7)

    for station_id in station_ids:
        # Construct URLs dynamically
        historical_url = f'https://fems.fs2c.usda.gov/api/climatology/download-nfdr?stationIds={station_id}&endDate=2024-12-31T23:30:00Z&startDate=2005-01-01T00:30:00Z&dataFormat=csv&dataset=all&fuelModels=Y'
        current_year_url = f'https://fems.fs2c.usda.gov/api/climatology/download-nfdr?stationIds={station_id}&endDate={today.strftime("%Y-%m-%d")}T00:30:00Z&startDate=2025-01-01T00:30:00Z&dataFormat=csv&dataset=all&fuelModels=Y'
        forecast_url = f'https://fems.fs2c.usda.gov/api/climatology/download-nfdr?stationIds={station_id}&endDate={seven_days_from_today.strftime("%Y-%m-%d")}T01:00:00Z&startDate={today.strftime("%Y-%m-%d")}T00:30:00Z&dataFormat=csv&dataset=all&fuelModels=Y'

        # Filenames for downloaded data
        historical_csv = f"historical_erc_{station_id}.csv"
        current_year_csv = f"current_year_erc_{station_id}.csv"
        forecast_csv = f"forecast_erc_{station_id}.csv"

        # Download datasets
        download_csv(historical_url, historical_csv)
        download_csv(current_year_url, current_year_csv)
        download_csv(forecast_url, forecast_csv)

        # Load and process datasets
        historical_df = load_and_process_csv(historical_csv)
        current_year_df = load_and_process_csv(current_year_csv)
        forecast_df = load_and_process_csv(forecast_csv)

        # Present the user with a menu to choose components
        print("Choose a component to visualize:")
        for i, column in enumerate(COLUMNS[4:], start=1):  # Skip non-measurement columns
            print(f"{i}. {column}")
        selected_component_index = input("Enter the number corresponding to your choice: ").strip()

        try:
            selected_component = COLUMNS[4:][int(selected_component_index) - 1]
        except (IndexError, ValueError):
            print("Invalid selection. Skipping to the next station.")
            continue

        # Calculate daily statistics
        historical_max, historical_avg = calculate_daily_stats(historical_df)
        current_max, _ = calculate_daily_stats(current_year_df)
        forecast_max, _ = calculate_daily_stats(forecast_df)

        # Calculate quantiles based on historical data
        quantiles = calculate_quantiles(historical_df, selected_component)

        # Plot the combined data
        plot_erc(historical_max, historical_avg, current_max, forecast_max, quantiles, station_id, selected_component)